In [ ]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil
import time


parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment

import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Температурная стабильность"
display(Markdown('### 💠 {}'.format(testname)))



In [ ]:
from connect_device_v2.find_device import FindDevice,parse_config
box = FindDevice()
display(box)

In [ ]:
button = box.select_device_continue_button

In [ ]:
%blockrun button

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
%blockrun button1

In [ ]:
inst = box.selected_device

In [ ]:
config =  parse_config(dict(box.config))
device = config['DEVICE']
test_time = float(device['time'])
step_time = float(device['step_time'])
ambient_temperature = device['ambient_temperature']
marker_range = ((config['RANGE']['start_stop'].replace(" ", "")).split(','))
start = []
stop = []
for freq in marker_range:
    start.append((freq.split('-'))[0])
    stop.append((freq.split('-'))[1])
segments = '5,0,0,1,1,0,18,300000,382643,13,0,0,382645,1e+06,19,0,0,1.1e+06,3.2e+06,11,0,0,3.3e+06,1e+07,11,0,0,1e+07,8.46475e+07,26,0,0,8.46475e+07,1.46285e+08,15,0,0,1.46285e+08,3.1567e+08,30,0,0,3.1567e+08,5.16284e+08,25,0,0,5.16284e+08,6.42568e+08,25,0,0,6.42568e+08,1.09653e+09,25,0,0,1.09653e+09,1.18333e+09,21,0,0,1.18333e+09,1.19871e+09,3,0,0,1.19871e+09,1.775e+09,21,0,0,1.775e+09,1.99871e+09,21,0,0,1.99871e+09,3.19871e+09,41,0,0,3.19871e+09,3.55e+09,21,0,0,3.55e+09,5.49871e+09,97,0,0,5.49871e+09,9e+09,175,0,0'
port_count = int(inst.query("SERV:PORT:COUN?"))

In [ ]:
date=datetime.now()
date=date.strftime('%Y-%m-%d %H:%M:%S')
device = inst.query("*IDN?")
device = device.replace(' ','')
device = device.split(',')
device_info = {'Время запуска':date,
              'device':device[0],
              'model':device[1],
              'serial':device[2],
              'version':device[3]}

In [ ]:
def trace_setting():
    inst.write(f"CALC:PAR:COUN {port_count}")
    for i in range(1,port_count+1):        
        inst.write(f"CALC:PAR{i}:DEF S{i}{i}")
        inst.write(f"CALC1:PAR{i}:SEL")
        inst.write(f"CALC:MARK{len(start)+1} 1")
        for marker in range(len(start)):
            inst.write(f"CALC:MARK{marker+1}:X {float(start[marker])*10**6}")
            check_error(inst)
        inst.write(f"CALC:MARK{len(start)+1}:X {float(stop[-1])*10**6}")
        check_error(inst)
        inst.write("CALC:MST 1")
        check_error(inst)
        inst.write("CALC:MST:DOM 1")
        check_error(inst)

In [ ]:
def get_start():
    inst.write("SYST:PRES")
    inst.write("TRIG:SOUR BUS")
    check_error(inst)
    inst.write("INIT:CONT OFF")
    check_error(inst)
    trace_setting()
    inst.write("SENS:BAND 100")
    check_error(inst)
    inst.write("SENS:SWE:TYPE SEGM")
    check_error(inst)
    inst.write(f"SENS:SEGM:DATA {segments}")
    check_error(inst)
    inst.write(f"SERVice:RFCTL:POWer:STATe 1")
    inst.write(f"SERVice:RFCTL:POWer:ATT 10")
    inst.write(f"SERV:RFCTL:POW:DAC 6554")
    disable_all_correctors(inst)
    check_error(inst)
    inst.timeout=150000


In [ ]:
def normalizeited_graf(trace_number):
    inst.write(f"CALC:PAR{trace_number}:SEL")
    check_error(inst)
    inst.write("CALC:MATH:MEM")
    check_error(inst)
    inst.write("CALC:MATH:FUNC DIVide")
    #inst.write("DISP:WIND:TRAC{}:MEM 0".format(n_graf))

In [ ]:
def start_scan():     
    inst.write("INIT")
    check_error(inst)
    inst.write("TRIG:SING")
    inst.query("*OPC?")
    check_error(inst)

In [ ]:
def get_markers_values(port,freq,data):
    inst.write(f"CALC1:PAR{port}:SEL")
    inst.write(f"CALC:MST:DOM:STAR {freq+1}")
    check_error(inst)
    inst.write(f"CALC:MST:DOM:STOP {freq+2}")
    check_error(inst)
    new = float(inst.query("CALC:MST:DATA?").split(',')[2])
    if new > float(data):
        return new
    else:
        return data

In [ ]:
def toFixed(numObj, digits=0):
        return f"{numObj:.{digits}f}"

In [ ]:
def test_step(step_time, step):
    temp_start = float(inst.query("SYST:TEMP:SENSor?").split('/')[0])
    result = {}
    result[f'{step}'] = {}
    start_time = time.time()
    start_scan()
    max_pp = []
    for i in range (1,port_count+1):
        normalizeited_graf(i)
        max_pp.append(int(0))
    for freq in range(len(start)):   
        result[f'{step}'][f'{start[freq]} - {stop[freq]}']={}
    while(time.time() - start_time < step_time*60):
        start_scan()
        for port in range(1,port_count+1):
            for freq in range(len(start)):
                max_pp[port-1] = get_markers_values(port,freq,max_pp[port-1])
                result[f'{step}'][f'{start[freq]} - {stop[freq]}'][f'S{port}{port}']= {'maximum': toFixed(max_pp[port-1], 4)}
    temp_stop = float(inst.query("SYST:TEMP:SENSor?").split('/')[0])
    temp_difference = temp_stop-temp_start
    for port in range(1,port_count+1):
            for freq in range(len(start)):
                drift = float(result[f'{step}'][f'{start[freq]} - {stop[freq]}'][f'S{port}{port}']['maximum'])
                if temp_stop != temp_start:
                    result[f'{step}'][f'{start[freq]} - {stop[freq]}'][f'S{port}{port}']['temp_stab'] =  toFixed(drift/temp_difference, 4)
                else:
                    result[f'{step}'][f'{start[freq]} - {stop[freq]}'][f'S{port}{port}']['temp_stab'] = 0
    return result, temp_stop

In [ ]:
def temp_stability():
    get_start()
    start_time = time.time()
    result_table = []
    temperature_table = []
    pbar = tqdm_notebook(total=test_time*60)
    oldprogress=(time.time()-start_time)

    while(time.time() - start_time < test_time*60):
        step = ((time.time() - start_time) // 60)
        result, temperature = test_step(step_time,step)
        result_table.append(result)
        temperature_table.append(temperature)
        progress=(time.time()-start_time)    
        pbar.update(progress-oldprogress)
        oldprogress=progress
    full_time = (time.time() - start_time) // 60
    return result_table, temperature_table, full_time

In [ ]:
result_table, temperature_table, full_time = temp_stability()

In [ ]:
def table_generator(table, temperature_table):
    step_time = []
    for i in table:
        for step in i.keys():
            step_time.append(step)
    range_marker = []
    for i in table[0]:
        for a in table[0][f'{i}']:
            range_marker.append(a)
            
    html_table=''
    colums_name=['Время','Темп.']
    for i in table[0].keys():
        for marker in table[0][f'{i}']:
            for trace in table[0][f'{i}'][f'{marker}']:
                colums_name.append(f"{trace} Дрейф, дБ")
                colums_name.append(f"{trace} Темп. стаб., дБ/град")
            break
            
    for marker in range_marker:
        html_table+='<table border="0"><tr>&nbsp</tr><tr></table>'
        html_table+='<table border="1">'              
        html_table+= f'<tr><td colspan="4">Диапазон {marker}, МГц</td></tr><tr>'
        for colums in colums_name:
            html_table+=f'<td>{colums}</td>'
        for index,step in enumerate(step_time):
            html_table+=f"</tr><tr><td>{step}</td>"
            html_table+=f'<td>{temperature_table[index]}</td>'
            for s in table[index].keys():
                for trace in table[index][f'{s}'][f'{marker}']:
                    html_table+=f"<td>{table[index][f'{s}'][f'{marker}'][f'{trace}']['maximum']}</td>"
                    html_table+=f"<td>{table[index][f'{s}'][f'{marker}'][f'{trace}']['temp_stab']}</td>"
        html_table+='</table>'                        
    return(html_table)

In [ ]:
def html_header(data, time):
    html_table='<table border="1"><tr>'
    for key in data:
        html_table+=f'<td>{key}</td>'
    html_table+=f'</tr>'
    html_table+=f'<tr>'
    for key in data:
        html_table+=f'<td>{data[key]}</td>'
    html_table+=f'</tr>'
    html_table+=f'<tr><td>Общее время теста:</td><td>{time}</td></tr>'
    html_table+=f'<tr><td>температура среды</td><td>{ambient_temperature}</td></tr></table>'
    return html_table

In [ ]:
from IPython.display import display, HTML
html_table=table_generator(result_table, temperature_table)
header = html_header(device_info, full_time)
display(HTML(header),HTML(html_table))



In [ ]:
file_data = html_header(device_info, full_time)
file_data += table_generator(result_table,temperature_table)
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook